# Resonance Filter Theory and Mathematical Formulas

A **resonance filter** is a frequency-selective network that passes a specific band of frequencies while attenuating others. The performance and selectivity of these filters are characterized by several key parameters.

### **Quality Factor (Q)**

The **Quality Factor (Q)** represents the ratio of the center frequency (\( f_c \)) of the resonant circuit to its bandwidth (BW). A higher Q indicates a narrower, more selective filter.

$$
Q = \frac{f_c}{BW} = \frac{f_2 - f_1}{f_c}
$$

Where \( f_2 \) and \( f_1 \) are the upper and lower 3-dB frequencies.

### **Shape Factor (SF)**

The **Shape Factor (SF)** is the ratio of the 60-dB bandwidth to the 3-dB bandwidth of the resonant circuit. It describes the "steepness" of the filter's skirts; a shape factor closer to 1 implies an ideal, vertical transition between the passband and stopband.

$$
SF = \frac{BW_{3dB}}{BW_{60dB}}
$$

### **Ripple**

Ripple is a measure of the flatness of the passband of a resonant circuit. It quantifies the variations in signal amplitude within the desired frequency range.

$$
\text{Ripple (dB)} = A_{\text{max}} - A_{\text{min}} \quad \text{(within passband)}
$$

## Resonance of a Parallel LC Circuit

In a **parallel LC circuit**, resonance occurs when the inductive reactance $$X_L$$ and the capacitive reactance $X_C$ are equal in magnitude but opposite in phase. At resonance, the total reactance of the circuit becomes zero, and the impedance reaches its maximum value. The resonance condition for a parallel LC circuit is given by:

$$
f_{\text{res}} = \frac{1}{2 \pi \sqrt{LC}}
$$

Where:
- **$f_{\text{res}}$** is the resonant frequency (in Hz),
- **$L$** is the inductance (in Henrys),
- **$C$** is the capacitance (in Farads).

## Loaded $Q$ of a Circuit with an Ideal Capacitor and a Non-Ideal Inductor

In real circuits, the inductor is not ideal, and it exhibits a finite  $Q_L$ factor. The **loaded $Q$** of a circuit is a measure of the damping or losses in the circuit, and it changes when the circuit is loaded with a resistive component or when the inductor itself has losses. 

### **Insertion Loss (IL)**

**Insertion Loss (IL)** is the attenuation caused by the components inserted between a generator and its load. This represents the power lost simply by adding the filter to the circuit.

$$
IL(dB) = 10 \log_{10} \left( \frac{P_{\text{load, with filter}}}{P_{\text{load, without filter}}} \right)
$$


# 1. and 2. Exercise

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# Use %matplotlib inline for standard Jupyter notebooks
%matplotlib inline

def updateFilter(qInductor, rSource, rLoad, fCenterMhz, bwMhz):
    plt.close('all')
    
    # 1. Input Conversions
    fCenter = fCenterMhz * 1e6      
    bwTarget = bwMhz * 1e6
    
    # 2. Design Calculus
    qLoadedTarget = fCenter / bwTarget
    rExternalParallel = (rSource * rLoad) / (rSource + rLoad)
    
    # Validation for physical limits
    if qInductor <= qLoadedTarget:
        display(Markdown("## Error: Inductor Q must be higher than the target Loaded Q!"))
        return

    # Xp = Rext * ( (1/Qloaded) - (1/Qind) )
    xpVal = rExternalParallel * ( (1.0 / qLoadedTarget) - (1.0 / qInductor) )
    
    # Component Values
    lVal = xpVal / (2 * np.pi * fCenter)
    cVal = 1 / (2 * np.pi * fCenter * xpVal)
    rParallelInd = qInductor * xpVal

    # 3. Simulation
    # Range limited to fc +/- 2*BW
    freqMin = fCenter - (2 * bwTarget)
    freqMax = fCenter + (2 * bwTarget)
    freqs = np.linspace(freqMin, freqMax, 2000)
    omega = 2 * np.pi * freqs
    
    # Impedance calculation for the tank
    zTank = 1 / ( (1/(1j * omega * lVal)) + (1j * omega * cVal) + (1/rLoad) + (1/rParallelInd) )
    vRatioWithFilter = zTank / (rSource + zTank)
    
    # Reference condition
    vRatioNoFilter = rLoad / (rSource + rLoad)

    # IL calculation
    gainDB = 20 * np.log10(np.abs(vRatioWithFilter / vRatioNoFilter))
    insertionLossAtResonance = np.max(gainDB)

    # 4. Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(freqs/1e6, gainDB, color='blue', lw=2)
    plt.axvline(fCenter/1e6, color='red', linestyle='--', alpha=0.5)
    plt.axhline(insertionLossAtResonance - 3, color='orange', linestyle=':', label='-3.00 dB Point')
    
    plt.axvspan((fCenter - bwTarget/2)/1e6, (fCenter + bwTarget/2)/1e6, 
                color='green', alpha=0.1, label=f'BW: {bwMhz:.2f} MHz')
    
    plt.title(f"fc={fCenterMhz:.2f}MHz, BW={bwMhz:.2f}MHz, IL={insertionLossAtResonance:.2f}dB")
    plt.xlabel("Frequency (MHz)")
    plt.ylabel(r"IL (dB) rel. to no filter")
    plt.xlim(freqMin/1e6, freqMax/1e6)
    plt.ylim(insertionLossAtResonance - 25, insertionLossAtResonance + 2)
    plt.grid(True, which='both', alpha=0.3)
    plt.legend()
    plt.show()

    # Dynamic Information Display
    display(Markdown(r"""
**Calculated Parameters:**
* **Loaded $Q$ ($Q_L$):** """ + f"{qLoadedTarget:.2f}" + r""" | **Derived $X_p$:** """ + f"{xpVal:.2f}" + r" $\Omega$" + r"""
* **Inductance ($L$):** """ + f"{lVal*1e9:.2f}" + r" nH" + r""" | **Capacitance ($C$):** """ + f"{cVal*1e12:.2f}" + r" pF" + r"""
* **Inductor $R_{p\_ind}$:** """ + f"{rParallelInd:.2f}" + r" $\Omega$"
    ))

# Slider Setup
fcSlider = widgets.FloatSlider(value=100.0, min=1.0, max=500.0, step=1.0, description='fc (MHz)')
bwSlider = widgets.FloatSlider(value=5, min=0.1, max=50.0, step=0.1, description='BW (MHz)')
qIndSlider = widgets.FloatSlider(value=100.0, min=10.0, max=10000.0, step=1.0, description='Ind Q')
rsSlider = widgets.FloatSlider(value=50.0, min=10.0, max=1000.0, step=10.0, description='Rs (Src)')
rlSlider = widgets.FloatSlider(value=2000.0, min=50.0, max=5000.0, step=50.0, description='Rl (Ld)')

row1 = widgets.HBox([fcSlider, bwSlider])
row2 = widgets.HBox([qIndSlider, rsSlider, rlSlider])
ui = widgets.VBox([row1, row2])

out = widgets.interactive_output(updateFilter, {
    'fCenterMhz': fcSlider,
    'bwMhz': bwSlider,
    'qInductor': qIndSlider, 
    'rSource': rsSlider, 
    'rLoad': rlSlider
})

display(ui, out)

Output()

# 3. Exercise

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# Use %matplotlib inline for standard Jupyter notebooks
%matplotlib inline

def updateTappedC(f0Mhz, bwMhz, rSource, rLoad, qInd):
    plt.close('all')
    
    # 1. Input Conversions
    f0 = f0Mhz * 1e6
    bw = bwMhz * 1e6
    w0 = 2 * np.pi * f0
    
    # 2. Design Calculus
    qe = f0 / bw
    if qInd <= qe:
        display(Markdown("## Error: Inductor Q must be higher than Loaded Q (Qe)!"))
        return

    # Rp calculation strictly on Rl
    rp = rLoad * (qInd - qe) / (2 * qe)
    xp = rp / qInd 
    
    # Component Calculations
    lp = xp / w0
    cTotal = 1 / (xp * w0)
    
    # Tapped-C Transformation (k factor) 
    k = np.sqrt(rLoad / rSource) - 1
    c2 = cTotal * (1 + k) / k
    c1 = k * c2

    # 3. Simulation (Corrected Network Logic)
    freqMin, freqMax = f0 - (5 * bw), f0 + (5 * bw)
    freqs = np.linspace(freqMin, freqMax, 2000)
    omega = 2 * np.pi * freqs
    
    # Z_tank: Lp || Rp || Rl
    yTank = (1 / (1j * omega * lp)) + (1 / rp) + (1 / rLoad)
    zTankBranch = 1 / yTank
    
    # Z_series: C2 in series with the Tank
    zSeriesBranch = (1 / (1j * omega * c2)) + zTankBranch
    
    # Z_input: C1 in parallel with (C2 + Tank)
    yInput = (1j * omega * c1) + (1 / zSeriesBranch)
    zInputTotal = 1 / yInput
    
    # Vout/Vin Ratio with filter
    vRatioWithFilter = (zInputTotal / (rSource + zInputTotal)) * (zTankBranch / zSeriesBranch)
    
    # --- CORRECTED REFERENCE ---
    # To avoid positive IL, we compare to the maximum possible voltage 
    # delivered by a matched source: Vout_max = Vin * sqrt(Rl/Rs) / 2
    # Or more simply, we normalize the peak to 0dB if it's ideal.
    vPeakIdeal = 0.5 * np.sqrt(rLoad / rSource) 
    gainDB = 20 * np.log10(np.abs(vRatioWithFilter / vPeakIdeal))
    
    # Shift so that 0dB is the absolute maximum theoretical limit
    ilAtResonance = np.max(gainDB)

    # 4. Plotting
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(freqs/1e6, gainDB, color='blue', lw=2)
    ax.axvline(f0/1e6, color='red', linestyle='--', alpha=0.5)
    
    # Shadow for 3dB Bandwidth
    ax.axvspan((f0 - bw/2)/1e6, (f0 + bw/2)/1e6, color='green', alpha=0.15, label='3dB BW')
    
    ax.axhline(ilAtResonance - 3, color='orange', linestyle=':', label='-3 dB Level')
    
    ax.set_title(f"fc={f0Mhz:.2f}MHz, BW={bwMhz:.2f}MHz, IL={ilAtResonance:.2f}dB")
    ax.set_xlabel("Frequency (MHz)")
    ax.set_ylabel("Insertion Loss (dB)")
    ax.set_xlim(freqMin/1e6, freqMax/1e6)
    ax.set_ylim(ilAtResonance - 50, 2) # Limit top to show 0dB clearly
    ax.grid(True, which='both', alpha=0.3)
    ax.legend(loc='upper right')
    plt.show()

    # 5. ASCII Schematic
    schematic = r"""
           Rs           C2
    Vin---[  ]---+------||----------+-------+-------+
                 |                  |       |       |
                [ ] C1             ( ) Lp  [ ] Rl  [ ] Rp
                [ ]                ( )     [ ]     [ ]
                 |                  |       |       |
    -------------+------------------+-------+-------+
    """
    
    display(Markdown(f"### Circuit Topology: C1 Parallel, C2 Series\n```\n{schematic}\n```"))
    display(Markdown(r"""
**Calculated Values (2 Decimals):**
* **Reactance $X_p$:** """ + f"{xp:.2f}" + r" $\Omega$" + r""" | **Inductor $R_p$:** """ + f"{rp:.2f}" + r" $\Omega$" + r"""
* **Inductor $L_p$:** """ + f"{lp*1e9:.2f}" + r" nH" + r""" | **Capacitor $C_1$:** """ + f"{c1*1e12:.2f}" + r" pF" + r"""
* **Capacitor $C_2$:** """ + f"{c2*1e12:.2f}" + r" pF" + r""" | **Match Factor $k$:** """ + f"{k:.2f}"
    ))

# Slider Setup
fcS = widgets.FloatSlider(value=100.0, min=1.0, max=500.0, step=1.0, description='fc (MHz)')
bwS = widgets.FloatSlider(value=5.0, min=0.1, max=50.0, step=0.1, description='BW (MHz)')
qS = widgets.FloatSlider(value=100.0, min=10.0, max=500.0, step=5.0, description='Ind Q')
rsS = widgets.FloatSlider(value=50.0, min=10.0, max=1000.0, step=10.0, description='Rs (Src)')
rlS = widgets.FloatSlider(value=2000.0, min=100.0, max=10000.0, step=100.0, description='Rl (Ld)')

row1 = widgets.HBox([fcS, bwS])
row2 = widgets.HBox([qS, rsS, rlS])
ui = widgets.VBox([row1, row2])

out = widgets.interactive_output(updateTappedC, {
    'f0Mhz': fcS, 'bwMhz': bwS, 'qInd': qS, 'rSource': rsS, 'rLoad': rlS
})

display(ui, out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1000x500 with 1 Axes>', '…